# Usage of `spm_encode`

This is an example of using spm_encode.

In [54]:
# Build the project
!mkdir -p build
%cd build
!cmake -D SPM_USE_EXTERNAL_ABSL=off -D SPM_ENABLE_TCMALLOC=on -D SPM_ENABLE_NFKC_COMPILE=on ..
# Alternatively, debug build:
# !cmake \
#       -DCMAKE_BUILD_TYPE=Debug -DCMAKE_C_FLAGS_DEBUG="-g -O0" \
#       -DCMAKE_CXX_FLAGS_DEBUG="-g -O0" \
#       -D SPM_USE_EXTERNAL_ABSL=off -D SPM_ENABLE_TCMALLOC=on -D SPM_ENABLE_NFKC_COMPILE=on ..
!make -j
%cd ..

/home/fsuser/haoyiyang/sentencepiece


/home/fsuser/haoyiyang/sentencepiece/build
-- VERSION: 0.2.00
-- Found the following ICU libraries:
--   i18n (required)
--   data (required)
--   uc (required)
-- Found TCMalloc: /usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so
-- Configuring done
-- Generating done
-- Build files have been written to: /home/fsuser/haoyiyang/sentencepiece/build
Consolidate compiler generated dependencies of target sentencepiece_train-static
Consolidate compiler generated dependencies of target sentencepiece
Consolidate compiler generated dependencies of target sentencepiece-static
[  9%] Built target sentencepiece_train-static
[ 70%] Built target sentencepiece-static
[ 79%] Built target sentencepiece
Consolidate compiler generated dependencies of target spm_encode
Consolidate compiler generated dependencies of target spm_decode
Consolidate compiler generated dependencies of target spm_bpe_cache_merge
Consolidate compiler generated dependencies of target spm_export_vocab
Consolidate compiler generated 

In [55]:
!printf 'Document with code\nline 1\n\1int x;\nfloat y;\2\nline 2\n\0\3File path: code.cpp\4\1int x;\nfloat y;\2\0' > example.in
!cat -A example.in

Document with code$
line 1$
^Aint x;$
float y;^B$
line 2$
^@^CFile path: code.cpp^D^Aint x;$
float y;^B^@

In [56]:
!./build/src/spm_encode --model=/home/fsuser/datasets/100k/pile_100k.sp.model --output_format=bid --input example.in --output example.bid --new_line_delim=0

spm_encode_main.cc(306) LOG(INFO) Encoded 2 sentences
spm_encode_main.cc(333) LOG(INFO) Encoded 2 sentences


In [57]:
with open("example.bid", "rb") as f:
    contents = f.read()
n_sentences = int.from_bytes(contents[-8:], "little")
print("n_sentences =", n_sentences)
sentence_sizes = []
for i in range(n_sentences):
    buffer = contents[-8 - (i+1) * 4: -8 - i*4]
    sentence_size = int.from_bytes(buffer, "little")
    sentence_sizes.append(sentence_size)
sentence_sizes = list(reversed(sentence_sizes))
print("sentence_sizes =", sentence_sizes)
ptr = 0
sentence_ids = []
for sentence_i, sentence_size in enumerate(sentence_sizes):
    buffer = contents[ptr: ptr + sentence_size * 4]
    ids = []
    for i in range(0, len(buffer), 4):
        ids.append(int.from_bytes(buffer[i: i + 4], "little"))
    sentence_ids.append(ids)
    print(f"sentence_ids[{sentence_i}] =", ids)
    ptr += sentence_size * 4

n_sentences = 2
sentence_sizes = [22, 17]
sentence_ids[0] = [13113, 356, 1868, 99340, 1183, 99320, 99347, 99340, 4, 555, 1166, 462, 8540, 357, 99391, 5, 678, 1183, 99320, 99351, 99340, 3]
sentence_ids[1] = [6, 7596, 1698, 99374, 1868, 99339, 12172, 7, 4, 555, 1166, 462, 8540, 357, 99391, 5, 3]


In [58]:
vocab_file = "/home/fsuser/datasets/100k/pile_100k.sp.vocab"
tokens = []
with open(vocab_file, "r", encoding="utf-8") as f:
    buffer = ""
    for line in f:
        buffer += line
        if "\t" in line:
            tokens.append(buffer[:buffer.index("\t")])
            buffer = ""
print("Load vocab size of", len(tokens))

Load vocab size of 100000


In [59]:
for ids in sentence_ids:
    print("=" * 20)
    print(" ".join(tokens[i] for i in ids))

▁Document ▁with ▁code 
 line ▁ 1 
 <0x01> ▁int ▁x ;
 float ▁y ; <0x02> ▁
 line ▁ 2 
 <0x00>
<0x03> ▁File ▁path : ▁code . cpp <0x04> <0x01> ▁int ▁x ;
 float ▁y ; <0x02> <0x00>
